This notebook gives the computational details of the results in the manuscript "Ruling Out Static Latent Homophily in Citation Networks" ([arXiv:1605.XXXXX](http://arxiv.org/abs/1605.XXXXX)). We are interested in studying causal structures in citation networks and their algebraic geometry. In particular, we study semidefinite programming relaxations (SDPs) of a latent homophily model.

We start with importing the necessary libraries. The module [metaknowledge](http://networkslab.org/metaknowledge/) handles the citation database and [Ncpol2sdpa](http://ncpol2sdpa.readthedocs.org/) for generating the SDP. It also needs at least one SDP solver; in the example below, we will use [Mosek](https://mosek.com/). Since metaknowledge requires Python 3, this notebook will not work with a Python 2 kernel.

In [ ]:
import metaknowledge as mk
import ncpol2sdpa as nc
import numpy as np
import re

We will need a handful of helper functions. The following are the $F_\pm$ and $S_\pm$ counters in Eq. (1):

$P(A_{1:T}|R_A)  =  \alpha_+^{F_+(A)}\alpha_-^{F_-(A)}(1-\alpha_-)^{S_-(A)}(1-\alpha_+)^{S_+(A)} \alpha_0^{1/2(1+A1)}(1-\alpha_0)^{1/2(1-A1)}$

We also define a function to generate chains of ${\pm}1$, and a class for storing the probabilities of the chains.

In [ ]:
def Fp(chain, T):
    return sum((1+chain[t])*(1-chain[t+1]*chain[t]) for t in range(T-1))//4


def Fn(chain, T):
    return sum((1-chain[t])*(1-chain[t+1]*chain[t]) for t in range(T-1))//4


def Sp(chain, T):
    return sum((1+chain[t])*(1+chain[t+1]*chain[t]) for t in range(T-1))//4


def Sn(chain, T):
    return sum((1-chain[t])*(1+chain[t+1]*chain[t]) for t in range(T-1))//4


def generate_chains(T, At=None, chain=None):
    if At is None:
        Atl = [1, -1]
    else:
        Atl = [At]
        self_chain = [c for c in chain]
    sum_ = []
    for Ai in Atl:
        if chain is None:
            self_chain = [Ai]
        else:
            self_chain.append(Ai)
        for Aj in [1, -1]:
            if T > 2:
                sum_ += generate_chains(T-1, Aj, self_chain)
            else:
                sum_.append(self_chain + [Aj])
    return sum_


class Probability(object):

    def __init__(self, T):
        chains = generate_chains(T)
        self.combinations = []
        for chain1 in chains:
            for chain2 in chains:
                self.combinations.append([chain1, chain2, 0])

    def __getitem__(self, chains):
        for combination in self.combinations:
            if combination[0] == chains[0] and combination[1] == chains[1]:
                return combination[2]
        raise Exception("Not found")

    def __setitem__(self, chains, value):
        for combination in self.combinations:
            if combination[0] == chains[0] and combination[1] == chains[1]:
                combination[2] = value
                return
        raise Exception("Not found")

    def normalize(self, Z):
        for combination in self.combinations:
            combination[2] /= Z

The next couple of functions work on the citation network. The first function attempts to mitigate the arrogance of Thomson-Reuters, who cannot be bothered to normalize author names. The second function picks all authors who made a reference in a time period. The third one defines a directed graph over the co-author network. The last one gets all references

In [ ]:
def normalize_name(name):
    w = re.sub("([a-z]{2,}) ", "\g<1>X", re.sub("[\.,]", " ", name.lower()))
    return w.replace(" ", "").replace("X", " ")


def get_authors_in_period(RC, year1, year2):
    RC_period = RC.yearSplit(year1, year2)
    authors_period = {}
    for R in RC_period:
        if R.citations is not None:
            reference_IDs = [reference.ID() for reference in R.citations]
            for author in R.authors:
                author = normalize_name(author)
                if author in authors_period:
                    authors_period[author] += reference_IDs
                else:
                    authors_period[author] = reference_IDs
    return authors_period


def create_directed_coauthor_graph(RC):
    coauthors_undirected = RC.coAuthNetwork()
    coauthors = coauthors_undirected.to_directed()
    weights = coauthors_undirected.degree()
    for edge in coauthors_undirected.edges_iter():
        if weights[edge[0]] > weights[edge[1]]:
            coauthors.remove_edge(edge[1], edge[0])
        else:
            coauthors.remove_edge(edge[0], edge[1])
    return coauthors


def get_all_references(author, epochs):
    references = set()
    for epoch in epochs:
        if author in epoch:
            for reference in epoch[author]:
                references.add(reference)
    return references

The following code snippet loads the database. Here we use the example set that comes with metaknowledge, and the commented out line would load the full corpus.

In [ ]:
RC = mk.RecordCollection("./savedrecs.txt")
# RC = mk.RecordCollection("./InfSci20JournBase.hci")

Next we split the corpus in three time periods and record the coauthor relationships in the first time period.

In [ ]:
years = []
for R in RC:
    if isinstance(R.year, int):
        years.append(R.year)
histogram = np.histogram(np.array(years), bins=3)
coauthors0 = create_directed_coauthor_graph(RC.yearSplit(histogram[1][0],
                                                         histogram[1][1]))

authors_period0 = get_authors_in_period(RC, histogram[1][0], histogram[1][1])
authors_period1 = get_authors_in_period(RC, histogram[1][1], histogram[1][2])
authors_period2 = get_authors_in_period(RC, histogram[1][2], histogram[1][3])
epochs = [authors_period0, authors_period1, authors_period2]

We populate the probability class with length-3 chains and add the statistics from the corpus across the three time periods:

In [ ]:
T = len(epochs)
p = Probability(T)
M = 0
for pair in coauthors0.edges():
    coauthor1 = normalize_name(pair[0])
    coauthor2 = normalize_name(pair[1])
    references = get_all_references(coauthor1, epochs)
    references.union(get_all_references(coauthor1, epochs))
    for reference in references:
        chain1, chain2 = [], []
        for epoch in epochs:
            if coauthor1 in epoch:
                if reference in epoch[coauthor1]:
                    chain1.append(+1)
                else:
                    chain1.append(-1)
            else:
                chain1.append(-1)
            if coauthor2 in epoch:
                if reference in epoch[coauthor2]:
                    chain2.append(+1)
                else:
                    chain2.append(-1)
            else:
                chain2.append(-1)
        p[(chain1, chain2)] += 1
        M += 1
p.normalize(M)

We set up the symbolic variables of the SDP relaxation, and set the moment constraints as outlined in Eq. (2), that is, 

$y_j = \sum_{R_A, R_B} P(R_A, R_B|E)f_j(x),$

where

$f_j(x) = \sum_{A,B}P(A_{1:T}|R_A)P(B_{1:T}|R_B)O_j(A,B).$

In [ ]:
A1 = 1
alpha0 = nc.generate_variables(name='alpha_0')[0]
alphap = nc.generate_variables(name='alpha_p')[0]
alpham = nc.generate_variables(name='alpha_m')[0]
beta0 = nc.generate_variables(name='beta_0')[0]
betap = nc.generate_variables(name='beta_p')[0]
betam = nc.generate_variables(name='beta_m')[0]
chains = generate_chains(T)
moments = []
for chain1 in chains:
    for chain2 in chains:
        if p[(chain1, chain2)] > 0:
            P_A = alphap**Fp(chain1, T) * alpham**Fn(chain1, T) * (1-alpham)**Sn(chain1, T) * (1-alphap)**Sp(chain1, T) * \
                    alpha0**((1+A1)//2) * (1-alpha0)**((1-A1)//2)
            P_B = betap**Fp(chain2, T) * betam**Fn(chain2, T) * (1-betam)**Sn(chain2, T) * (1-betap)**Sp(chain2, T) * \
                    beta0**((1+A1)//2) * (1-beta0)**((1-A1)//2)
            moments.append(P_A*P_B - p[(chain1, chain2)])

We also have to make sure that the constraints Eq. (3) are satisfied to ensure that we have a valid probability distribution"

$x\in\mathbb{R}^6: g_i(x)=x_i(1-x_i)\geq 0, i=1,\ldots,6.$

These constraints are not on the moments but on the actual variables, so matching localizing matrices will be generated in the SDP relaxation.

In [ ]:
inequalities = [alpha0, alphap, alpham, beta0, betap, betam,
                1-alpha0, 1-alphap, 1-alpham, 1-beta0, 1-betap, 1-betam]

Finally, we generate a level-3 relaxation, and solve it as a feasibility problem.

In [ ]:
sdp = nc.SdpRelaxation([alpha0, alphap, alpham, beta0, betap, betam])
sdp.get_relaxation(3, inequalities=inequalities, momentequalities=moments)
sdp.solve(solver="mosek")

If we study the result, for the example set that comes with metaknowledge, the status will be optimal. In this case, we cannot say anything about static latent homophily. Executing the notebook with the actual database used in the paper gives an infeasible status, which allows us to reject the hypothesis.

In [ ]:
print(sdp.primal, sdp.dual, sdp.status)